In [1]:
import csv
import os
import threading
import requests
import tweepy
from dotenv import load_dotenv
import webbrowser

In [2]:
MAX_SIZE = 265000000   # size of output file: 265MB
CONSUMER_KEY = "9CZOMsHKv1YKmaatoaFVs67l1"   # provided key from twitter for developer app
CONSUMER_SECRET = "inUbtMG0n4pVMZEllzwneC5cxZTAQwTq28Kf0hwsXfzmKHMuub"   # provided key from twitter for developer app
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)   # authorize app access to twitter account

True

In [3]:
pin = input("Enter pin for twitter account access: ")
auth.get_access_token(pin)   # get access tokens for developer account
ACCESS_TOKEN = auth.access_token
ACCESS_SECRET = auth.access_token_secret
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

Enter pin for twitter account access: 9228194


In [4]:
def download(link, filelocation):
    try:
        r = requests.get(link, stream=True)
        if not os.path.exists(os.path.dirname(filelocation)):
            try:
                os.makedirs(os.path.dirname(filelocation))
            except OSError as exc: # Guard against race condition
                pass
        with open(filelocation, 'wb') as f:
            for chunk in r.iter_content(1024):
                if chunk:
                    f.write(chunk)
    except Exception as e:
        print(e)

In [5]:
class CustomStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        urls = [link['expanded_url'] for link in status.entities['urls']]
        for url in urls:
            filename = url
            filename = filename.replace('https://', '')
            filename = filename.replace('http://', '')
            filename = filename.replace('/', '_')
            filelocation = os.path.join('downloads', str(status.id), filename)
            download_thread = threading.Thread(target=download, args=(url, filelocation))
            download_thread.start()
        with open('output.csv', 'a', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([
                status.id,
                status.author.screen_name,
                status.created_at,
                status.text,
                status.place,
                ['#{}'.format(hashtag['text']) for hashtag in status.entities['hashtags']],
                urls
            ])
            if f.tell() > MAX_SIZE:
                return False

    def on_error(self, status_code):
        print('error {}'.format(status_code))

In [6]:
streamingAPI = tweepy.streaming.Stream(auth, CustomStreamListener())
streamingAPI.filter(track=['Biden', 'President', 'Kamala Harris' 'United States'], is_async=True)

[Errno 22] Invalid argument: 'downloads\\1361516172584361993\\www.knopnews2.com_2021_02_15_biden-faces-pressure-as-us-sets-new-course-on-immigration_?utm_campaign=snd-autopilot&utm_source=twitter&utm_medium=social&utm_campaign=snd&utm_content=knop'
[Errno 22] Invalid argument: 'downloads\\1361516111708102660\\www.washingtonpost.com_business_2021_02_15_biden-stimulus-state-local-aid_?tid=ss_tw'
[Errno 22] Invalid argument: 'downloads\\1361516205064941574\\twitter.com_atrupar_status_1133757374966239233?s=19'
